In [25]:
import pandas as pd
import pyperclip
from IPython.display import display, HTML

def get_urls(season, game_nos, print_details=False, area="liv"):
    results = pd.read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", parse_dates = ["game_date"], usecols = ["season", "ssn_game_no", "game_date", "opposition", "venue", "competition", "score"])

    if type(game_nos) == list:
        min_game_no = min(game_nos)
        max_game_no = max(game_nos)
    else:
        min_game_no = game_nos
        max_game_no = game_nos

    print(len("ALL MATCHES") * "-")
    print("ALL MATCHES")
    print(len("ALL MATCHES") * "-")
    results = results[(results.season == season) & (results.ssn_game_no >= min_game_no) & (results.ssn_game_no <= max_game_no)].drop(columns = ["season", "ssn_game_no"]).sort_values(by = "game_date")

    display(HTML(results.to_html(index = False)))

    dates = results.game_date

    for date in enumerate(dates):
        n_date = date[0] + 1
        date = date[1]
        
        title = f"Match {n_date} of {len(dates)}"
        print(len(title) * "-")
        print(title)
        print(len(title) * "-")
        
        result = results[results.game_date == date]

        display(HTML(result.to_html(index = False)))

        date_1 = results[results.game_date == date].game_date.astype(str).values[0]
        date_2 = results[results.game_date == date].game_date + pd.Timedelta(days=5)
        date_2 = date_2.astype(str).values[0]

        opp = results[results.game_date == date].opposition.str.split(' ').str[0].values[0]
        
        if area == "liv":
            url = f"https://www.britishnewspaperarchive.co.uk/search/results/{date_1}/{date_2}?basicsearch=%2btranmere%20%2b{opp}&freesearch=tranmere%20{opp}&place=liverpool%2c%20lancashire%2c%20england&retrievecountrycounts=false&mostspecificlocation=liverpool%2c%20lancashire%2c%20england&sortorder=dayearly"
        else:
            url = f"https://www.britishnewspaperarchive.co.uk/search/results/{date_1}/{date_2}?basicsearch=%2btranmere%20%2b{opp}&freesearch=tranmere%20{opp}&&retrievecountrycounts=false&sortorder=dayearly"

        if print_details:
            sub_mins = pd.read_csv("./manual/subs_missing_mins.csv", parse_dates = ["game_date"])
            sub_mins = sub_mins[sub_mins.game_date == date]
            
            print("\nSUBSTITUTES")
            print(len("SUBSTITUTES") * "-")
            if len(sub_mins) == 0:
                print("No subs.\n")
            else:
                display(HTML(sub_mins.to_html(index = False)))

            yc = pd.read_csv("./manual/cards_yellow.csv", parse_dates = ["game_date"])
            yc = yc[yc.game_date == date]
            print("\nYELLOW CARDS")
            print(len("Yellow cards") * "-")
            display(HTML(yc.to_html(index = False)))

            rc = pd.read_csv("./manual/cards_red.csv", parse_dates = ["game_date"])
            rc = rc[rc.game_date == date]
            print("\nRED CARDS")
            print(len("Red cards") * "-")
            display(HTML(rc.to_html(index = False)))

            goal_mins = pd.read_csv("./manual/goal_mins_new.csv", parse_dates = ["game_date"])
            goal_mins = goal_mins[goal_mins.game_date == date_1]
            print("\nGOAL MINS")

            display(HTML(goal_mins.to_html(index = False)))

        print("\nLIBRARY URL:")
        print(len("Library URL:") * "-")
        print(f"{url}\n\n")

        if len(dates) == 1:
            pyperclip.copy(url)

In [31]:
get_urls("1996/97", [32, 32])

-----------
ALL MATCHES
-----------


game_date,opposition,venue,score,competition
1997-01-14,Carlisle United,A,0-1,FA Cup


------------
Match 1 of 1
------------


game_date,opposition,venue,score,competition
1997-01-14,Carlisle United,A,0-1,FA Cup



LIBRARY URL:
------------
https://www.britishnewspaperarchive.co.uk/search/results/1997-01-14/1997-01-19?basicsearch=%2btranmere%20%2bCarlisle&freesearch=tranmere%20Carlisle&place=liverpool%2c%20lancashire%2c%20england&retrievecountrycounts=false&mostspecificlocation=liverpool%2c%20lancashire%2c%20england&sortorder=dayearly




In [5]:
def save_files():
    sub_mins = pd.read_csv("./manual/subs_missing_mins.csv", parse_dates = ["game_date"]).sort_values(by = ["game_date"])
    sub_mins.to_csv("./manual/subs_missing_mins.csv", index = False)

    yc = pd.read_csv("./manual/cards_yellow.csv", parse_dates = ["game_date"]).sort_values(by = ["game_date"])
    yc.to_csv("./manual/cards_yellow.csv", index = False)

    rc = pd.read_csv("./manual/cards_red.csv", parse_dates = ["game_date"]).sort_values(by = ["game_date", "min_so"])
    rc.to_csv("./manual/cards_red.csv", index = False)

    goal_mins = pd.read_csv("./manual/goal_mins_new.csv", parse_dates = ["game_date"]).sort_values(by = ["game_date", "goal_min"])
    goal_mins.to_csv("./manual/goal_mins_new.csv", index = False)

save_files()